<a href="https://colab.research.google.com/github/yg-kim-korean/fastcampusAI/blob/main/fastcampus_%EB%94%A5%EB%9F%AC%EB%8B%9D_%EC%8B%A4%EC%A0%84_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!nvidia-smi

Wed Feb 17 01:33:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [28]:
#라이브러리 임포트
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import re
from PIL import Image
import shutil
import random
import matplotlib.pyplot as plt


In [29]:
print(tf.__version__)
print(keras.__version__)

2.4.1
2.4.0


In [30]:
import gdown
url = 'https://drive.google.com/uc?id=1dIR9ANjUsV9dWa0pS9J0c2KUGMfpIRG0'
fname = 'oxford_pet.zip'
gdown.download(url, fname, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1dIR9ANjUsV9dWa0pS9J0c2KUGMfpIRG0
To: /content/oxford_pet.zip
811MB [00:06, 126MB/s]


'oxford_pet.zip'

In [31]:
## Oxford_pet.zip이 보이는지 확인
!ls -l

total 791584
drwx------ 4 root root      4096 Feb 17 00:38 drive
drwxr-xr-x 4 root root      4096 Feb 17 00:38 oxford_pet
-rw-r--r-- 1 root root 810565619 Feb 17 01:34 oxford_pet.zip
drwxr-xr-x 1 root root      4096 Feb 10 14:40 sample_data


In [32]:
##압축풀기
!unzip -q oxford_pet.zip -d oxford_pet

replace oxford_pet/annotations/._trimaps? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
All


In [33]:
##압축이풀린 directory 확인
!ls -l

total 791584
drwx------ 4 root root      4096 Feb 17 00:38 drive
drwxr-xr-x 4 root root      4096 Feb 17 00:38 oxford_pet
-rw-r--r-- 1 root root 810565619 Feb 17 01:34 oxford_pet.zip
drwxr-xr-x 1 root root      4096 Feb 10 14:40 sample_data


In [34]:
##디렉터리 설정
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, 'oxford_pet')
image_dir = os.path.join(data_dir, 'images')

In [35]:
#image file 수 확인
images_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] =='.jpg']
print(len(images_files))

7390


In [41]:
## image file 들을 읽어서 channel이 3이 아닌 image는 삭제
for images_file in images_files:
  image_path = os.path.join(image_dir, images_file)
  image = Image.open(image_path)
  image_mode = image.mode
  if image_mode !='RGB':
    print(images_file, image_mode)
    image = np.asarray(image)
    print(image.shape)
    os.remove(image_path)

In [42]:
images_files[:10]

['Ragdoll_118.jpg',
 'Persian_194.jpg',
 'american_pit_bull_terrier_69.jpg',
 'german_shorthaired_60.jpg',
 'Siamese_83.jpg',
 'Ragdoll_132.jpg',
 'german_shorthaired_128.jpg',
 'scottish_terrier_73.jpg',
 'wheaten_terrier_197.jpg',
 'chihuahua_159.jpg']

In [38]:
#image file 수 확인
images_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] =='.jpg']
print(len(images_files))

7378


In [43]:
class_list = set()
for image_file in images_files:
  file_name = os.path.splitext(image_file)[0]
  class_name = re.sub('_\d+', '',file_name)
  class_list.add(class_name)
class_list = list(class_list)
print(len(class_list))

37


In [44]:
class_list.sort()
class_list

['Abyssinian',
 'Bengal',
 'Birman',
 'Bombay',
 'British_Shorthair',
 'Egyptian_Mau',
 'Maine_Coon',
 'Persian',
 'Ragdoll',
 'Russian_Blue',
 'Siamese',
 'Sphynx',
 'american_bulldog',
 'american_pit_bull_terrier',
 'basset_hound',
 'beagle',
 'boxer',
 'chihuahua',
 'english_cocker_spaniel',
 'english_setter',
 'german_shorthaired',
 'great_pyrenees',
 'havanese',
 'japanese_chin',
 'keeshond',
 'leonberger',
 'miniature_pinscher',
 'newfoundland',
 'pomeranian',
 'pug',
 'saint_bernard',
 'samoyed',
 'scottish_terrier',
 'shiba_inu',
 'staffordshire_bull_terrier',
 'wheaten_terrier',
 'yorkshire_terrier']

In [45]:
class_list[1]

'Bengal'

In [47]:
class2idx = {cls:idx for idx, cls in enumerate(class_list)}
class2idx

{'Abyssinian': 0,
 'Bengal': 1,
 'Birman': 2,
 'Bombay': 3,
 'British_Shorthair': 4,
 'Egyptian_Mau': 5,
 'Maine_Coon': 6,
 'Persian': 7,
 'Ragdoll': 8,
 'Russian_Blue': 9,
 'Siamese': 10,
 'Sphynx': 11,
 'american_bulldog': 12,
 'american_pit_bull_terrier': 13,
 'basset_hound': 14,
 'beagle': 15,
 'boxer': 16,
 'chihuahua': 17,
 'english_cocker_spaniel': 18,
 'english_setter': 19,
 'german_shorthaired': 20,
 'great_pyrenees': 21,
 'havanese': 22,
 'japanese_chin': 23,
 'keeshond': 24,
 'leonberger': 25,
 'miniature_pinscher': 26,
 'newfoundland': 27,
 'pomeranian': 28,
 'pug': 29,
 'saint_bernard': 30,
 'samoyed': 31,
 'scottish_terrier': 32,
 'shiba_inu': 33,
 'staffordshire_bull_terrier': 34,
 'wheaten_terrier': 35,
 'yorkshire_terrier': 36}

In [48]:
class2idx['Bengal']

1

In [49]:
##train, validation directory 생성
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [50]:
images_files.sort()

In [51]:
images_files[:10]

['Abyssinian_1.jpg',
 'Abyssinian_10.jpg',
 'Abyssinian_100.jpg',
 'Abyssinian_101.jpg',
 'Abyssinian_102.jpg',
 'Abyssinian_103.jpg',
 'Abyssinian_104.jpg',
 'Abyssinian_105.jpg',
 'Abyssinian_106.jpg',
 'Abyssinian_107.jpg']

In [52]:
cnt=0
previous_class = ''
for image_file in images_files:
  file_name = os.path.splitext(image_file)[0]
  class_name = re.sub('_\d+', '',file_name)
  if class_name == previous_class:
    cnt +=1
  else:
    cnt = 1
  if cnt<= 160:
    cpath = train_dir
  else:
    cpath = val_dir
  image_path = os.path.join(image_dir, image_file)
  shutil.copy(image_path, cpath)
  previous_class = class_name
  

In [53]:
train_images = os.listdir(train_dir)
val_images = os.listdir(val_dir)

In [54]:
print(len(train_images), len(val_images))

5920 1458


In [55]:
train_images[:10]

['Ragdoll_118.jpg',
 'Persian_194.jpg',
 'german_shorthaired_60.jpg',
 'Ragdoll_132.jpg',
 'german_shorthaired_128.jpg',
 'wheaten_terrier_197.jpg',
 'chihuahua_159.jpg',
 'shiba_inu_205.jpg',
 'Sphynx_155.jpg',
 'Russian_Blue_29.jpg']

In [56]:
val_images[:10]

['american_pit_bull_terrier_69.jpg',
 'Siamese_83.jpg',
 'scottish_terrier_73.jpg',
 'beagle_63.jpg',
 'american_pit_bull_terrier_76.jpg',
 'boxer_89.jpg',
 'miniature_pinscher_96.jpg',
 'chihuahua_93.jpg',
 'Birman_85.jpg',
 'Ragdoll_74.jpg']

#TFRecord File 만들기